In [1]:
import ray
#ray.shutdown()
#ray.init()

In [3]:
from files import BCTrade_utils
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()
name ='Bitstamp_2'
dataset = Dataset.get_by_name(ws, name=name)
env = BCTrade_utils.BCTrade(dataset.to_pandas_dataframe(),singlerun=False,spreadfactor=0.0)
experiment_name = BCTrade_utils.registerBCTrade(name)
from ray.tune.registry import register_env
def env_creator(env_config):
    return BCTrade_utils.BCTrade(dataset.to_pandas_dataframe())
register_env("BCTrade-Bitstamp_2", env_creator)

In [4]:
#datastore = ws.get_default_datastore()
#checkpoint_ds = Dataset.File.from_files(path=[(datastore,'checkpoints_BCTrade-Btc_with_indicators_1641664748_e3baadf3_head/checkpoint_370')])
#checkpoint = checkpoint_ds.as_named_input('test')
#checkpoint.as_mount('xxx')
#checkpoint = '/tmp/checkpoint_370/checkpoint-370'
#Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_100
checkpoint = '/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/'
checkpoint+='logs/PPO/PPO_BCTrade-Bitstamp_1_c06f9bf6_2022-02-15_19-16-18we4ohh4i/checkpoint_277/checkpoint-277'

#checkpoint = 'logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_110/checkpoint-110'

In [5]:
import ray
import ray.rllib.agents.ppo as ppo

#ray.init()
#config = {
  #"callbacks": {
  #  "on_train_result": "<function on_train_result at 0x7fca737a8e18>"
  #},
#  "env": "BCTrade-Btc_with_indicators",
#  "num_gpus": 0,
#  "num_workers": 1
#}

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
#config["eager"] = False
config["env"] = "BCTrade-Bitstamp_2"
new_trainer = ppo.PPOTrainer(config=config,env="BCTrade-Bitstamp_2")
new_trainer


2022-02-15 21:05:10,952	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=25120) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=25120)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=25120) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(RolloutWorker pid=25120) Instructions for updating:
(RolloutWorker pid=25120) If using Keras pass *_constraint arguments to layers.
(RolloutWorker pid=25120) 2022-02-15 21:05:36,645	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


PPO

In [6]:
import pickle
with open(checkpoint, "rb") as f:
        extra_data = pickle.load(f)

extra_data['worker']
weights=pickle.loads(extra_data['worker'])
weights['state']['default_policy']
new_trainer.get_policy().set_weights(weights['state']['default_policy'])
#new_trainer.get_policy().set_state(weights['state'])
#new_trainer.restore(checkpoint)
#ray.tune.run(ppo.PPOTrainer, config=config, restore='/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_95fc9bdc_2022-01-10_21-38-0176w5xe80/checkpoint_822')

In [7]:
#obs = env.initialize()
obs = env.reset()
rewardsum=0
c=0
while True:
	action = new_trainer.compute_action(obs, explore=False)
	if obs[0]==1 and action ==2:
		env.render()
	obs, reward, done, _ = env.step(action)
	rewardsum+=reward
	c+=1
	if done:
		break
env.render()
print(rewardsum,rewardsum/c,c)


2022-02-15 21:08:02,933	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


1002.4299287410925
1003.0807600950119
1001.6462404165096
1002.4021413620129
1003.885323398873
1004.7870810858211
1005.0637551223865
1005.0589870076406
1005.4930460691342
1007.3565302232291
1007.5092478565604
1005.2829345932771
1005.2757864568306
1003.7984769122907
1003.039545866651
1003.761499883321
1004.498558333466
1001.1761634076217
1001.8069760738259
1000.038258326832
998.686585621053
998.2322825528473
997.8784182203483
996.9725118835308
998.7117106783526
998.6142122852513
998.1115513399238
997.8196114724852
996.6845411573754
998.1726480344165
997.5306706139129
998.1746235812528
998.1746235812528
999.6630453375675
1000.4690756496931
999.4403022376518
997.3019741294825
997.3019741294825
996.1552925445793
996.2027246711473
995.4657288331508
995.4657288331508
995.4657288331508
996.4150879086301
997.8692866573608
997.7414257313806
998.6162839546416
999.8355840709272
999.5747118215118
998.6609153071666
998.6609153071666
998.6609153071666
998.6609153071666
1000.9412179444357
1000.1596186